In [1]:
import numpy as np # Linear algebra
import pandas as pd # for working with databases
import os # for reading multiple files

In [2]:
path = '/kaggle/input/decemberstationsdatakoenig'
files = os.listdir(path)
print(len(files), "files found")

31 files found


In [3]:
dfs = []

for file in files:
    if file.endswith(".csv"):
        df_temp = pd.read_csv(os.path.join(path, file))
        dfs.append(df_temp)

df = pd.concat(dfs, ignore_index=True)

print("Size of splited DataFrame:", df.shape)

Size of splited DataFrame: (548580, 11)


In [4]:
df.head()

,uuid,name,brand,street,house_number,post_code,city,latitude,longitude,first_active,openingtimes_json
0,44e2bdb7-13e3-4156-8576-8326cdd20459,bft Tankstelle,NaN,Schellengasse,53,36304,Alsfeld,50.752009,9.279039,1970-01-01 01:00:00+01,{}
1,0e18d0d3-ed38-4e7f-a18e-507a78ad901d,OIL! Tankstelle München,OIL!,Eversbuschstraße 33,NaN,80999,München,48.180700,11.460900,1970-01-01 01:00:00+01,"{""openingTimes"":[{""applicable_days"":192,""perio..."
2,ad812258-94e7-473d-aa80-d392f7532218,bft Bonn-Bad Godesberg,bft,Godesberger Allee,55,53175,Bonn,50.695100,7.142760,1970-01-01 01:00:00+01,"{""overrides"":[{""startp"":""2025-12-31 05:00"",""en..."
3,e8d55212-b30f-449e-b65c-913a7b2002b1,Esso Tankstelle,ESSO,KONRAD-ADENAUER-STR. 32,,72762,REUTLINGEN,48.492396,9.202997,2014-03-18 16:45:31+01,{}
4,e8dbc04a-b775-42e1-bcbd-8550e239ed55,OIL! Tankstelle Hamburg,OIL!,Rolfinckstraße 48,NaN,22391,Hamburg,53.639300,10.089200,2014-03-18 16:45:31+01,"{""openingTimes"":[{""applicable_days"":224,""perio..."


In [5]:
df = df.drop(columns=['first_active', 'openingtimes_json'])

In [6]:
df = df.drop_duplicates(subset='uuid', keep='last')

In [7]:
print(df["brand"].value_counts())

brand
ARAL                                   2458
Shell                                  1881
ESSO                                   1305
TotalEnergies                           785
AVIA                                    726
                                       ... 
 Freie Tankstelle Hirschmann              1
Tankstelle & Espressobar Nardi            1
R. Tesche GmbH                            1
Tank und Wasch                            1
Freie Tankstelle Autohaus Hörl GmbH       1
Name: count, Length: 1168, dtype: int64


In [8]:
df["brand"] = (
    df["brand"]
    .str.lower()                                  # все в нижній регістр
    .str.replace("-", " ")                        # замінити дефіс на пробіл
    .str.replace(r"[^a-z0-9\s]", "", regex=True)  # видалити все крім букв, цифр і пробілів
    .str.strip()                                  # обрізати пробіли з країв
    .str.replace(r"\s+", " ", regex=True)         # замінити подвійні пробіли
)

In [9]:
top_brands = (
    df["brand"]
    .value_counts()
    .head(60)
)

print(top_brands)

brand
aral                       2458
shell                      1882
esso                       1390
totalenergies               785
avia                        739
jet                         716
bft                         509
agip eni                    502
star                        496
raiffeisen                  468
hem                         430
freie tankstelle            321
oil                         230
sb                          216
avia xpress                 204
classic                     183
westfalen                   167
supermarkt tankstelle       157
q1                          132
hoyer                       125
sprint                      120
baywa                       112
ed                          109
team                        105
orlen                        91
elan                         85
total                        84
freie                        83
bft walther                  75
nordoel                      71
access                       70
pm

In [10]:
df["tokens"] = df["brand"].fillna("").str.split()


In [11]:
from collections import Counter

counter = Counter()

df["tokens"].apply(counter.update)

counter.most_common(150)

[('aral', 2472),
 ('shell', 1883),
 ('esso', 1390),
 ('tankstelle', 1047),
 ('avia', 958),
 ('totalenergies', 877),
 ('jet', 716),
 ('bft', 699),
 ('raiffeisen', 599),
 ('freie', 557),
 ('agip', 533),
 ('eni', 510),
 ('star', 496),
 ('hem', 430),
 ('sb', 338),
 ('oil', 285),
 ('supermarkt', 240),
 ('xpress', 227),
 ('gmbh', 204),
 ('classic', 191),
 ('energie', 176),
 ('westfalen', 175),
 ('hoyer', 146),
 ('q1', 136),
 ('sprint', 120),
 ('baywa', 112),
 ('ed', 110),
 ('orlen', 110),
 ('tank', 107),
 ('team', 106),
 ('markant', 89),
 ('pm', 89),
 ('elan', 86),
 ('total', 85),
 ('walther', 83),
 ('lanfer', 73),
 ('markt', 72),
 ('nordoel', 71),
 ('access', 70),
 ('tankcenter', 68),
 ('eg', 63),
 ('score', 59),
 ('t', 58),
 ('calpam', 58),
 ('autohof', 57),
 ('globus', 57),
 ('frei', 56),
 ('warenhaus', 52),
 ('gulf', 50),
 ('roth', 48),
 ('greenline', 48),
 ('24h', 43),
 ('tankpoint', 42),
 ('omv', 41),
 ('famila', 41),
 ('mtb', 40),
 ('autohaus', 40),
 ('am', 38),
 ('ts', 38),
 ('kg', 3

In [12]:
BRAND_KEYWORDS = {
    "aral": "aral",
    "shell": "shell",
    "esso": "esso",
    "avia": "avia",
    "totalenergies": "total",
    "total": "total",
    "jet": "jet",
    "bft": "bft",
    "raiffeisen": "raiffeisen",
    "agip": "eni", # agip is eni
    "eni": "eni",
    "star": "star",
    "hem": "hem",
    "hoyer": "hoyer",
    "q1": "q1",
    "sprint": "sprint",
    "baywa": "baywa",
    "orlen": "orlen",
    "team": "team",
    "classic": "classic",
    "westfalen": "westfalen",
    "markant": "markant",
    "calpam": "calpam",
    "elan": "elan",
    "nordoel": "nordoel",
    "globus": "globus",
    "gulf": "gulf",
    "omv": "omv",
    "famila": "famila",
    "kaufland": "kaufland",
    "edeka": "edeka",
    "rewe": "rewe",
    "allguth": "allguth",
    "tinq": "tinq",
    "tamoil": "tamoil",
    "avanti": "avanti",
    "ran": "ran",
}

In [13]:
def normalize_brand(tokens):
    for token in tokens:
        if token in BRAND_KEYWORDS:
            return BRAND_KEYWORDS[token]
    return None  # якщо не знайдено бренд

In [14]:
df["normalized_brand"] = df["tokens"].apply(normalize_brand)

In [15]:
df.head()

,uuid,name,brand,street,house_number,post_code,city,latitude,longitude,tokens,normalized_brand
70802,57a8a929-175c-4cbe-86f7-834646e33f0d,MBG Freie Tankstelle,NaN,Zeppelinstraße,66-68,88045,Friedrichshafen,47.656737,9.453704,[],None
424713,8dcd43a6-0ef3-4dcf-88b4-82f7c1d5038f,Mengin Tank-Stop Gladenbach,mengin,Marburger Strasse,30,35075,Gladenbach,8.588450,5.077378,[mengin],None
530880,44e2bdb7-13e3-4156-8576-8326cdd20459,bft Tankstelle,NaN,Schellengasse,53,36304,Alsfeld,50.752009,9.279039,[],None
530881,0e18d0d3-ed38-4e7f-a18e-507a78ad901d,OIL! Tankstelle München,oil,Eversbuschstraße 33,NaN,80999,München,48.180700,11.460900,[oil],None
530882,ad812258-94e7-473d-aa80-d392f7532218,bft Bonn-Bad Godesberg,bft,Godesberger Allee,55,53175,Bonn,50.695100,7.142760,[bft],bft


In [16]:
df = df.drop(columns=["tokens"])

In [17]:
print((df['latitude'] == 0).sum())

33


In [18]:
df['latitude'] = df['latitude'].replace(0, np.nan)
print((df['latitude'] == 0).sum())

0


In [19]:
print((df['longitude'] == 0).sum())

33


In [20]:
df['longitude'] = df['longitude'].replace(0, np.nan)
print((df['longitude'] == 0).sum())

0


In [21]:
import geopandas as gpd
from shapely.geometry import Point

In [22]:
gdf_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.longitude, df.latitude),
    crs="EPSG:4326"  # WGS84, стандарт для координат
)

In [23]:
gadm_lands = gpd.read_file("/kaggle/input/gadm41/gadm41_DEU_4.json")
gadm_lands = gadm_lands.to_crs("EPSG:4326")  # перевіряємо, що CRS співпадає

In [24]:
joined = gpd.sjoin(gdf_points, gadm_lands, how="left", predicate="within")

In [25]:
joined.head()

,uuid,name,brand,street,house_number,post_code,city,latitude,longitude,normalized_brand,...,NAME_1,GID_2,NAME_2,GID_3,NAME_3,NAME_4,VARNAME_4,TYPE_4,ENGTYPE_4,CC_4
70802,57a8a929-175c-4cbe-86f7-834646e33f0d,MBG Freie Tankstelle,NaN,Zeppelinstraße,66-68,88045,Friedrichshafen,47.656737,9.453704,None,...,Baden-Württemberg,DEU.1.6_1,Bodenseekreis,DEU.1.6.2_1,Friedrichshafen,Friedrichshafen,NA,Stadt,Town,084355002016
424713,8dcd43a6-0ef3-4dcf-88b4-82f7c1d5038f,Mengin Tank-Stop Gladenbach,mengin,Marburger Strasse,30,35075,Gladenbach,8.588450,5.077378,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
530880,44e2bdb7-13e3-4156-8576-8326cdd20459,bft Tankstelle,NaN,Schellengasse,53,36304,Alsfeld,50.752009,9.279039,None,...,Hessen,DEU.7.22_1,Vogelsbergkreis,DEU.7.22.1_1,Alsfeld,Alsfeld,NA,Stadt,Town,065350001001
530881,0e18d0d3-ed38-4e7f-a18e-507a78ad901d,OIL! Tankstelle München,oil,Eversbuschstraße 33,NaN,80999,München,48.180700,11.460900,None,...,Bayern,DEU.2.54_1,München(KreisfreieStadt),DEU.2.54.1_1,München,München,NA,Stadt,Town,091620000000
530882,ad812258-94e7-473d-aa80-d392f7532218,bft Bonn-Bad Godesberg,bft,Godesberger Allee,55,53175,Bonn,50.695100,7.142760,bft,...,Nordrhein-Westfalen,DEU.10.3_1,Bonn,DEU.10.3.1_1,Bonn,Bonn,NA,Stadt,Town,053140000000


In [26]:
joined.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 17707 entries, 70802 to 548579
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   uuid              17707 non-null  object  
 1   name              17706 non-null  object  
 2   brand             17030 non-null  object  
 3   street            17705 non-null  object  
 4   house_number      16680 non-null  object  
 5   post_code         17706 non-null  object  
 6   city              17704 non-null  object  
 7   latitude          17674 non-null  float64 
 8   longitude         17674 non-null  float64 
 9   normalized_brand  12914 non-null  object  
 10  geometry          17707 non-null  geometry
 11  index_right       17655 non-null  float64 
 12  GID_4             17655 non-null  object  
 13  GID_0             17655 non-null  object  
 14  COUNTRY           17655 non-null  object  
 15  GID_1             17655 non-null  object  
 16  NAME_1        

In [27]:
joined.to_csv('cleaned_tankstellen.csv', index=False)